### Drop upc / Create dummy variables for DD2 and FN

In [2]:
train = pd.read_csv("data/train_v2.csv", index_col=0)

In [3]:
train.tail()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,DepartmentDescription_2,FinelineNumber
647049,39,191346,Sunday,3.239000e+10,1,PHARMACY OTC,pharmacy,1118.0
647050,39,191346,Sunday,7.874205e+09,1,FROZEN FOODS,food_1,1752.0
647051,39,191346,Sunday,4.072000e+03,1,PRODUCE,food_1,4170.0
647052,8,191347,Sunday,4.190008e+09,1,DAIRY,food_1,1512.0
647053,8,191347,Sunday,3.800060e+09,1,GROCERY DRY GOODS,food_1,3600.0


In [4]:
train['DepartmentDescription_2'].value_counts()

food_1           288582
household         52493
hygiene           46553
others            30102
pharmacy          26228
food_2            19179
deco              17916
baby              16697
hygene            14865
kids              14515
food_3            13669
men               12588
women             12557
service           11781
pets               9761
gardening          8162
kitchen            7600
office             6578
accessories        6220
accessaries        6099
electronics        5995
sport_outdoor      5947
auto               5414
entertainment      2985
interior           2784
home_garden        1720
tmp                  64
Name: DepartmentDescription_2, dtype: int64

In [5]:
# Weekday integer로..
def convert_weekday(day):
    if day == 'Sunday':
        return 0
    elif day == 'Monday':
        return 1
    elif day == 'Tuesday':
        return 2
    elif day == 'Wednesday':
        return 3
    elif day == 'Thursday':
        return 4
    elif day == 'Friday':
        return 5
    else:
        return 6
    
train.Weekday = train.Weekday.apply(convert_weekday)

In [6]:
# Returns column 추가
train['Returns'] = pd.Series([abs(num) if num < 0 else 0 for num in train.ScanCount], index=train.index)

In [7]:
# ScanCount rename Purchases
train.ScanCount = train.ScanCount.apply(lambda x: 0 if x < 0 else x)
train.rename(columns={'ScanCount': 'Purchases'}, inplace=True)

In [8]:
train.isnull().sum()

TripType                      0
VisitNumber                   0
Weekday                       0
Upc                        4129
Purchases                     0
DepartmentDescription      1361
DepartmentDescription_2       0
FinelineNumber             4129
Returns                       0
dtype: int64

In [9]:
train.drop(['Upc'], inplace=True, axis=1)
train['DepartmentDescription'] = train['DepartmentDescription'].fillna('None')
train['FinelineNumber'] = train['FinelineNumber'].fillna(-1)

In [10]:
dum_DD = pd.get_dummies(train.DepartmentDescription_2).astype(int, copy=False)
dum_FN = pd.get_dummies(train.FinelineNumber).astype(int, copy=False)

In [1]:
dum_FN.info()

NameError: name 'dum_FN' is not defined

In [9]:
# 일단 보류
# dum_DD.drop(dum_DD.columns[0], inplace=True, axis=1)
# dum_FN.drop(dum_FN.columns[0], inplace=True, axis=1)

In [16]:
train.drop(['DepartmentDescription', 'DepartmentDescription_2', 'FinelineNumber'], inplace=True, axis=1)

In [17]:
train.tail()

,TripType,VisitNumber,Weekday,Purchases,Returns
647049,39,191346,0,1,0
647050,39,191346,0,1,0
647051,39,191346,0,1,0
647052,8,191347,0,1,0
647053,8,191347,0,1,0


In [18]:
train.astype(int, copy=False)
train_2 = pd.concat([train, dum_DD, dum_FN], axis=1)

In [19]:
train_2.head()

,TripType,VisitNumber,Weekday,Purchases,Returns,accessaries,accessories,auto,baby,deco,...,9964.0,9966.0,9967.0,9970.0,9971.0,9974.0,9975.0,9991.0,9997.0,9998.0
0,999,5,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,7,5,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,7,5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26,8,5,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,26,8,5,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_2.to_csv("data/train_3.csv")